In [1]:
import pandas as pd

In [12]:
data = "/home/mdafifal.mamun/notebooks/triagerX/data/deeptriage/mozilla_core/deep_data.csv"

df = pd.read_csv(data).sample(1000, random_state=42)

In [13]:
df.head()

,owner,issue_title,description
114949,nobody@mozilla.org,createLinearGradient not displaying,"In test.html, the call to fillRect does not pr..."
107177,nobody@mozilla.org,Percentage height calculation of content in a ...,User-Agent: Mozilla/4.0 (compatible; MSI...
33261,john@johnkeiser.com,Inconsistent behavior when assigning values to...,From Bugzilla Helper:\nUser-Agent: Mozilla/5.0...
130814,nobody@mozilla.org,Youzeek App crashes,Steps to reproduce:\n1) install Youzeek App\n2...
92917,kaie@kuix.de,data: channel carrying text/html causes secure...,From bug 450912 comment 39:\n\nInstall NoScrip...


In [16]:
df["text"] = df.apply(lambda row: str(row["issue_title"]) + "\n" + str(row["description"]), axis=1)

In [17]:
texts = df["text"].to_list()

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Example text data
# texts = ["Sample text A", "Another example text B", "Another example text B", "Computer programming", "Else"]
inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling

KeyboardInterrupt: 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(embeddings)
silhouette_avg = silhouette_score(embeddings, clusters)
print(f"Silhouette Score: {silhouette_avg}")


Silhouette Score: 0.37491530179977417


In [1]:
from triagerx.model.triagerx_dev_model import TriagerxDevModel

In [2]:
model = TriagerxDevModel(42, 3, 3, ["microsoft/deberta-base", "roberta-base"], 0.2, max_tokens=256, num_filters=256, label_map={1: "test"})

2024-06-08 22:14:31.571 | DEBUG    | triagerx.model.triagerx_dev_model:__init__:24 - Loading base transformer models: ['microsoft/deberta-base', 'roberta-base']
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
params = sum(p.numel() for p in model.parameters())

In [4]:
params

294056583

In [5]:
from triagerx.model.cnn_transformer import CNNTransformer
from triagerx.model.fcn_transformer import FCNTransformer

In [8]:
model2 = CNNTransformer(986, -1, 1, 0.2, "microsoft/codebert-base", max_tokens=256, num_filters=256, label_map=None)

2024-10-21 13:36:33.607 | DEBUG    | triagerx.model.cnn_transformer:__init__:29 - Initiating full training...


In [9]:
params2 = sum(p.numel() for p in model2.parameters())
params2

129198298

In [10]:
model3 = FCNTransformer(986, -1, 0.2, "microsoft/codebert-base", 256, None)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-10-21 13:36:52.465 | DEBUG    | triagerx.model.fcn_transformer:__init__:31 - Initiating full training...


In [11]:
params3 = sum(p.numel() for p in model3.parameters())
params3

125403866

In [27]:
params2 / params

1.4264504665076654

In [12]:
import torch

torch.save(model, "test1.pth")
torch.save(model2, "test2.pth")

In [18]:
model2.load_state_dict(torch.load("test2.pth"))

TypeError: Expected state_dict to be dict-like, got <class 'triagerx.model.cnn_transformer.CNNTransformer'>.

In [ ]:
model2.get_label_map